## Import Libraries

In [1]:
import requests
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import pycountry
from plotly.subplots import make_subplots
from bs4 import BeautifulSoup
from plotly.offline import plot

#This is a test for Github

#This is a second test for Github

#somethign new


## Perform Web scrapping with BeautifulSoup

In [2]:
github = 'https://github.com'
URL = 'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports'

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

latest = soup.find_all('a')[-21]['href']

latestURL = github + latest
latestPage = requests.get(latestURL, 'html.parser')

soup = BeautifulSoup(latestPage.content, 'html.parser')
raw = str(soup.find(id='raw-url')['href'])
rawURL = github + raw
print(rawURL)

https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_daily_reports/11-24-2020.csv


## Create DataFrame
This df contains daily reports (Confirmed, Deaths, Recovered and Active). Also drop the rows (axis=0) where the Lat value is missing and fill up the fields with 0 where data is missing. Lastly get the data for the last update


In [9]:
df = pd.read_csv(rawURL)
print(df.head())
df = df.dropna(axis=0, subset=['Lat'])
df = df.fillna(0)
date, time = df.iloc[0,4].split()
df[0:100]


   FIPS Admin2 Province_State Country_Region          Last_Update       Lat  \
0   NaN    NaN            NaN    Afghanistan  2020-11-25 05:27:20  33.93911   
1   NaN    NaN            NaN        Albania  2020-11-25 05:27:20  41.15330   
2   NaN    NaN            NaN        Algeria  2020-11-25 05:27:20  28.03390   
3   NaN    NaN            NaN        Andorra  2020-11-25 05:27:20  42.50630   
4   NaN    NaN            NaN         Angola  2020-11-25 05:27:20 -11.20270   

       Long_  Confirmed  Deaths  Recovered   Active Combined_Key  \
0  67.709953      45280    1712      36122   7446.0  Afghanistan   
1  20.168300      34300     735      16666  16899.0      Albania   
2   1.659600      77000    2309      50070  24621.0      Algeria   
3   1.521800       6351      76       5503    772.0      Andorra   
4  17.873900      14742     338       7444   6960.0       Angola   

   Incident_Rate  Case_Fatality_Ratio  
0     116.316285             3.780919  
1    1191.882688             2.14285

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,0.0,0,0,Afghanistan,2020-11-25 05:27:20,33.93911,67.709953,45280,1712,36122,7446.0,Afghanistan,116.316285,3.780919
1,0.0,0,0,Albania,2020-11-25 05:27:20,41.15330,20.168300,34300,735,16666,16899.0,Albania,1191.882688,2.142857
2,0.0,0,0,Algeria,2020-11-25 05:27:20,28.03390,1.659600,77000,2309,50070,24621.0,Algeria,175.594455,2.998701
3,0.0,0,0,Andorra,2020-11-25 05:27:20,42.50630,1.521800,6351,76,5503,772.0,Andorra,8219.763153,1.196662
4,0.0,0,0,Angola,2020-11-25 05:27:20,-11.20270,17.873900,14742,338,7444,6960.0,Angola,44.854499,2.292769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0.0,0,Biobio,Chile,2020-11-25 05:27:20,-37.44640,-72.141600,34936,578,32757,1601.0,"Biobio, Chile",2244.083235,1.654454
100,0.0,0,Coquimbo,Chile,2020-11-25 05:27:20,-29.95900,-71.338900,12982,264,12601,117.0,"Coquimbo, Chile",1713.600832,2.033585
101,0.0,0,Los Lagos,Chile,2020-11-25 05:27:20,-41.91980,-72.141600,19735,180,18362,1193.0,"Los Lagos, Chile",2381.417821,0.912085
102,0.0,0,Los Rios,Chile,2020-11-25 05:27:20,-40.23100,-72.331100,5623,46,4956,621.0,"Los Rios, Chile",1461.138092,0.818069


## Map 1
This map show Confirmed cases, Deaths and Recovered. It is 3 maps in one.

In [10]:
mean = df['Confirmed'].mean()
normlised_data_C = [value/df['Confirmed'].mean() + 10 for value in  df['Confirmed']]
normlised_data_D = [value/df['Confirmed'].mean()+3 if value != 0  else value for value  in df['Deaths']]
normlised_data_R = [value/df['Confirmed'].mean() + 3 if value != 0  else value for value  in df['Recovered']]

hoverdata1 = df['Country_Region'] + " - "+ ['Confirmed cases: ' + str(v) for v in df['Confirmed'].tolist()]
hoverdata2 = df['Country_Region'] + " - "+ ['Death: ' + str(v) for v in df['Deaths'].tolist()]
hoverdata3 = df['Country_Region'] + " - "+ ['Recovered: ' + str(v) for v in df['Recovered'].tolist()]
#text = df['Country/Region']
fig_map1 = make_subplots()
#df = pd.read_csv('03-03-2020.csv')
fig_map1_1 = go.Figure(data=go.Scattergeo(
        lon = df['Long_'],
        lat = df['Lat'],
    name = 'Confirmed cases',
        hovertext = hoverdata1,
        marker = dict(
            size =  normlised_data_C,
            opacity = 0.5,
            color = 'blue',
            line = dict(
                width=0,
                color='rgba(102, 102, 102)'
            ),
        ),
        ))

fig_map1_2 = go.Figure(data=go.Scattergeo(
        lon = df['Long_'],
        lat = df['Lat'],
    name = 'Deaths',
        hovertext = hoverdata2,
        marker = dict(
            size =  normlised_data_D,
            opacity = 0.5,
            color = 'red',
            line = dict(
                width=0,
                color='rgba(102, 102, 102)'
            ),
        ),
        ))


fig_map1_3= go.Figure(data=go.Scattergeo(
        lon = df['Long_'],
        lat = df['Lat'],
        hovertext = hoverdata3,
     name = 'Recovered',
        marker = dict(
            size =  normlised_data_R,
            opacity = 0.5,
            color = 'green',
            line = dict(
                width=0,
                color='rgba(102, 102, 102)'
            ),
        ),
        ))

fig_map1.add_trace(fig_map1_1.data[0])
fig_map1.add_trace(fig_map1_2.data[0])
fig_map1.add_trace(fig_map1_3.data[0])

fig_map1.update_layout(
        title = f'The global impact of COVID-19.  Date: {date}',
    legend=dict(
        itemsizing = "constant",
        font=dict(
            family="sans-serif",
            size=20,
            color="black"
        )
    )
)
plot(fig_map1)

'temp-plot.html'

## Map 2
This map focuses on Deaths but it can be changed. Test 'mapbox_layers' to see another theme for the map

In [11]:
date, time = df.iloc[0,4].split()
#print(date,time)
fig_map2 = px.scatter_mapbox(
    df, lat="Lat", lon="Long_",
    color="Deaths",
    size="Deaths",
    color_continuous_scale=px.colors.sequential.Jet,
    size_max=60,
    zoom=3, 
    center={"lat": 37.0902, "lon": -95.7129},              
    hover_data=['Province_State'],
    mapbox_style='open-street-map',
    hover_name='Country_Region'
)
fig_map2.update_layout(
    title=f'COVID-19 Deaths in USA by states.  Date: {date}',
    #mapbox_layers=[
        #{
       #     "below": 'traces',
      #      "sourcetype": "raster",
     #       "sourceattribution": "United States Geological Survey",
    #        "source": [
   #             "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
  #          ]
 #       }
#      ]
)
plot(fig_map2)

'temp-plot.html'

## Pie Chart 1
This pie chart focuses on Confirmed cases

In [4]:
fig_pie1 = px.pie(df, values='Confirmed', names='Country_Region',color_discrete_sequence=px.colors.sequential.Plasma_r,
             title='Infected rates across countries',
              hover_data=[])
fig_pie1.update_traces(textposition='inside')
plot(fig_pie1)

'temp-plot.html'

## Bar Chart 1
This bar chart focuses on Confirmed cases

In [7]:
fig_bar1 = px.bar(df, x='Country_Region', y = 'Confirmed', color='Country_Region',hover_data=['Province_State'], title ='Number of reported Confirmed cases across countries')
plot(fig_bar1)

'temp-plot.html'

## Tree Map 1
This threeMap focuses on Recovered cases. Note that USA is missing.

In [8]:
fig_treeMap1 = px.treemap(df, path=['Country_Region'], values='Recovered', height=600, width=1000)

fig_treeMap1.update_layout(
    title="Confirmed cases",
    title_x = 0.05,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))

plot(fig_treeMap1)

'temp-plot.html'

## Get time series data
Create Data Frames for Confirmed cases, Deaths and Recovered. 

In [14]:
github = 'https://github.com'
URL = 'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series'

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)

latest1 = soup.find_all('a')[-29]['href']
latest2 = soup.find_all('a')[-25]['href']
latest3 = soup.find_all('a')[-23]['href']
print(latest1)
print(latest2)
print(latest3)


latestURL1 = github + latest1
latestURL2 = github + latest2
latestURL3 = github + latest3

latestPage1 = requests.get(latestURL1, 'html.parser')
latestPage2 = requests.get(latestURL2, 'html.parser')
latestPage3 = requests.get(latestURL3, 'html.parser')

soup1 = BeautifulSoup(latestPage1.content, 'html.parser')
soup2 = BeautifulSoup(latestPage2.content, 'html.parser')
soup3 = BeautifulSoup(latestPage3.content, 'html.parser')

raw1 = str(soup1.find(id='raw-url')['href'])
raw2 = str(soup2.find(id='raw-url')['href'])
raw3 = str(soup3.find(id='raw-url')['href'])

rawURL1 = github + raw1
rawURL2 = github + raw2
rawURL3 = github + raw3

print(rawURL1)
print(rawURL2)
print(rawURL3)

country_cases_file_link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv"


/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv
/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv
/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv
https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv
https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv
https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv


In [15]:
df_ts_Confirmed = pd.read_csv(rawURL1)
df_ts_Deaths=  pd.read_csv(rawURL2)
ds_ts_recovered =  pd.read_csv(rawURL3)
cases_country_df =  pd.read_csv(country_cases_file_link)

df_ts_Confirmed.head()
#df_ts_Deaths.head()
#ds_ts_recovered.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,42795,42969,43035,43240,43403,43628,43851,44228,44443,44503
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,26211,26701,27233,27830,28432,29126,29837,30623,31459,32196
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,65108,65975,66819,67679,68589,69591,70629,71652,72755,73774
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,5616,5725,5725,5872,5914,5951,6018,6066,6142,6207
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,13053,13228,13374,13451,13615,13818,13922,14134,14267,14413


### Clean the data and generate one exmple on how to call one specific country

In [16]:
df_ts_Confirmed = df_ts_Confirmed.replace(np.nan,'',regex = True)
df_ts_Deaths = df_ts_Deaths.replace(np.nan,'',regex = True)
ds_ts_recovered = ds_ts_recovered.replace(np.nan,'',regex = True)
cases_country_df = cases_country_df.replace(np.nan,'',regex = True)

df_ts_Confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20
0,,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,42795,42969,43035,43240,43403,43628,43851,44228,44443,44503
1,,Albania,41.15330,20.168300,0,0,0,0,0,0,...,26211,26701,27233,27830,28432,29126,29837,30623,31459,32196
2,,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,65108,65975,66819,67679,68589,69591,70629,71652,72755,73774
3,,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,5616,5725,5725,5872,5914,5951,6018,6066,6142,6207
4,,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,13053,13228,13374,13451,13615,13818,13922,14134,14267,14413


In [17]:
df_ts_Confirmed[df_ts_Confirmed['Country/Region']=='Australia']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,...,114,114,114,114,114,115,115,115,115,115
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,4469,4469,4486,4498,4502,4509,4514,4517,4527,4538
10,Northern Territory,Australia,-12.4634,130.8456,0,0,0,0,0,0,...,42,46,46,46,46,46,46,46,46,46
11,Queensland,Australia,-27.4698,153.0251,0,0,0,0,0,0,...,1183,1185,1185,1185,1186,1187,1190,1190,1192,1193
12,South Australia,Australia,-34.9285,138.6007,0,0,0,0,0,0,...,517,517,522,544,547,551,551,553,554,555
13,Tasmania,Australia,-42.8821,147.3272,0,0,0,0,0,0,...,230,230,230,230,230,230,230,230,230,230
14,Victoria,Australia,-37.8136,144.9631,0,0,0,0,1,1,...,20345,20345,20345,20345,20345,20345,20345,20345,20345,20345
15,Western Australia,Australia,-31.9505,115.8605,0,0,0,0,0,0,...,776,776,783,787,788,794,794,796,797,799


### Get global data (aggregate all the values for confirmed, recovered and deaths)

In [18]:
cases_country_df.head()

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Afghanistan,2020-11-22 20:27:01,33.9391,67.71,44706.0,1687.0,35934,7085.0,114.842,,,3.773543,4,AFG
1,Albania,2020-11-22 20:27:01,41.1533,20.1683,32761.0,699.0,15842,16220.0,1138.4,,,2.133635,8,ALB
2,Algeria,2020-11-22 20:27:01,28.0339,1.6596,74862.0,2272.0,48794,23796.0,170.719,,,3.034918,12,DZA
3,Andorra,2020-11-22 20:27:01,42.5063,1.5218,6256.0,76.0,5358,822.0,8096.81,,,1.214834,20,AND
4,Angola,2020-11-22 20:27:01,-11.2027,17.8739,14493.0,337.0,7346,6810.0,44.0969,,,2.325260,24,AGO


In [19]:
global_data = cases_country_df.copy().drop(['Lat','Country_Region','Last_Update','Long_','Incident_Rate','People_Tested','People_Hospitalized','Mortality_Rate','UID','ISO3'],axis=1)
global_summary = pd.DataFrame(global_data.sum()).transpose()
global_summary

,Confirmed,Deaths,Active
0,58483683.0,1385499.0,19713309.0


## Add ISO codes to df_TimeSeries_Confirmed_Global
This might not be neccesary but just in case

In [20]:
def get_alpha (name, alpha) :
  try :
    if alpha == 2 :
      return  pycountry.countries.get(name = name).alpha_2
    if alpha == 3 :
      return pycountry.countries.get(name = name).alpha_3
  except :
    return None

df_ts_Confirmed['country_iso3'] = df_ts_Confirmed['Country/Region'].apply(lambda x : get_alpha(x,3) )
df_ts_Deaths['country_iso3'] = df_ts_Confirmed['Country/Region'].apply(lambda x : get_alpha(x,3) )
ds_ts_recovered['country_iso3'] = df_ts_Confirmed['Country/Region'].apply(lambda x : get_alpha(x,3) )



In [21]:
df_ts_Confirmed2 = df_ts_Confirmed.groupby([ 'Country/Region','country_iso3', ]).aggregate('sum').drop(['Lat', 'Long'], axis =1).reset_index()
df_ts_Confirmed2.head()

,Country/Region,country_iso3,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20
0,Afghanistan,AFG,0,0,0,0,0,0,0,0,...,42795,42969,43035,43240,43403,43628,43851,44228,44443,44503
1,Albania,ALB,0,0,0,0,0,0,0,0,...,26211,26701,27233,27830,28432,29126,29837,30623,31459,32196
2,Algeria,DZA,0,0,0,0,0,0,0,0,...,65108,65975,66819,67679,68589,69591,70629,71652,72755,73774
3,Andorra,AND,0,0,0,0,0,0,0,0,...,5616,5725,5725,5872,5914,5951,6018,6066,6142,6207
4,Angola,AGO,0,0,0,0,0,0,0,0,...,13053,13228,13374,13451,13615,13818,13922,14134,14267,14413


In [22]:
Confirmed_ts = df_ts_Confirmed2.copy().drop(['Country/Region','country_iso3'], axis=1)
Confirmed_ts_summary = Confirmed_ts.sum()
Confirmed_ts_summary.head()

1/22/20     552
1/23/20     649
1/24/20     932
1/25/20    1425
1/26/20    2104
dtype: int64

## Line Chart 1 [Confirmed]

This is a time series line chart for Confirmed cases

In [38]:
fig_ts_lineChart = go.Figure(data = go.Scatter(x = Confirmed_ts_summary.index,
                                    y = Confirmed_ts_summary.values, mode = 'lines+markers'))
fig_ts_lineChart.update_layout(title = "Total Coronavirus Confirmed Cases (Globally)",
                   yaxis_title="Confirmed cases", xaxis_tickangle = 315)

plot(fig_ts_lineChart)

'temp-plot.html'

In [24]:
color_arr = px.colors.qualitative.Dark24

In [32]:
def draw_plot(tickangle=0,*,ts_array, ts_label, title, colors, mode_size, line_size, x_axis_title, y_axis_title, yaxis_type,additional_annotations):
    fig = go.Figure()
    
    for index, ts in enumerate(ts_array):
        fig.add_trace(go.Scatter(x=ts.index,
                                 y=ts.values,
                                 name=ts_label[index],
                                 line=dict(color=colors[index], width=line_size[index]),connectgaps=True,))
        
    x_axis_dict = dict(showline=True,
                      showgrid=True,
                      showticklabels=True,
                      linecolor='rgb(204, 204, 204)',
                      linewidth=2,
                      ticks='outside',
                      tickfont=dict(family='Arial', size=12, color='rgb(82, 82, 82)',))

    if x_axis_title:
        x_axis_dict['title'] = x_axis_title

    if tickangle > 0:
        x_axis_dict['tickangle'] = tickangle

    y_axis_dict = dict(showline = True,
                      showgrid = True,
                      showticklabels=True,
                      linecolor='rgb(204, 204, 204)',
                      linewidth=2,)

    if yaxis_type != " ":
            y_axis_dict['type'] = yaxis_type

    if y_axis_title:
        y_axis_dict['title'] = y_axis_title

    fig.update_layout(xaxis = x_axis_dict,
                     yaxis = y_axis_dict,
                     autosize=True,
                     margin=dict(autoexpand=True,l=100,r=20,t=110,),
                     showlegend=True,)

    annotations = []

    annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05, xanchor='left', yanchor='bottom',
                           text=title,
                           font=dict(family='Arial', size=16,color='rgb(37,37,37)'),showarrow=False))

    if len(additional_annotations)>0:
        annotations.append(additional_annotations)

    fig.update_layout(annotations=annotations)

    return fig
    

In [33]:
confirmed_agg_ts = df_ts_Confirmed.copy().drop(['Lat','Long','Country/Region','Province/State'], axis=1).sum()
death_agg_ts = df_ts_Deaths.copy().drop(['Lat','Long','Country/Region','Province/State'], axis=1).sum()
recovered_agg_ts=ds_ts_recovered.copy().drop(['Lat','Long','Country/Region','Province/State'], axis=1).sum()

active_agg_ts = pd.Series(
data = np.array(
[x1 - x2 - x3 for (x1, x2, x3) in zip(confirmed_agg_ts.values,death_agg_ts.values, recovered_agg_ts.values)]),
index = confirmed_agg_ts.index)

death_agg_ts

1/22/20          17
1/23/20          18
1/24/20          26
1/25/20          42
1/26/20          56
             ...   
11/17/20    1338106
11/18/20    1349380
11/19/20    1360381
11/20/20    1372182
11/21/20    1380436
Length: 305, dtype: int64

In [34]:
ts_array = [confirmed_agg_ts, active_agg_ts, recovered_agg_ts, death_agg_ts]
labels = ['Confirmed', 'Active', 'Recovered', 'Deaths']
colors = [color_arr[5], color_arr[0], color_arr[2], color_arr[3]]
mode_size = [8,8,12,8]
line_size = [2,2,4,2]

fig = draw_plot(ts_array = ts_array,
                 ts_label = labels,
                 title = 'Covid-19 Case Status(22nd Jan to 28th March 2020)',
                 colors = colors, mode_size = mode_size,
                 line_size = line_size,
                 x_axis_title = 'Date',
                 y_axis_title = 'Case Count',
                 tickangle = 315,
                 yaxis_type = ' ',additional_annotations=[])

plot(fig)

'temp-plot.html'

In [23]:
cases_country_df.head()

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Afghanistan,2020-11-22 00:26:56,33.9391,67.71,44503.0,1675.0,35422,7406.0,114.32,,,3.763791,4,AFG
1,Albania,2020-11-22 00:26:56,41.1533,20.1683,32196.0,685.0,15469,16042.0,1118.77,,,2.127593,8,ALB
2,Algeria,2020-11-22 00:26:56,28.0339,1.6596,73774.0,2255.0,48183,23336.0,168.238,,,3.056632,12,DZA
3,Andorra,2020-11-22 00:26:56,42.5063,1.5218,6207.0,76.0,5290,841.0,8033.39,,,1.224424,20,AND
4,Angola,2020-11-22 00:26:56,-11.2027,17.8739,14413.0,336.0,7273,6804.0,43.8535,,,2.331229,24,AGO


In [24]:
cases_country_df.copy().drop(
['Lat','Long_','Last_Update','Incident_Rate','People_Tested','People_Hospitalized','Mortality_Rate','UID','ISO3'],axis = 1).sort_values("Confirmed", ascending=False).reset_index(drop=True).style.bar(align="left",width=98,color='#d65f5f')

,Country_Region,Confirmed,Deaths,Recovered,Active
0,US,12059686.000000,255800.000000,4529700.000000,7274186.000000
1,India,9050597.000000,132726.000000,8478124.000000,439747.000000
2,Brazil,6020164.000000,168613.000000,5430896.000000,420655.000000
3,France,2178023.000000,48593.000000,156755.000000,1972675.000000
4,Russia,2047563.000000,35442.000000,1564897.000000,447224.000000
5,Spain,1556730.000000,42619.000000,150376.000000,1363735.000000
6,United Kingdom,1497129.000000,54721.000000,3310.000000,1439098.000000
7,Italy,1380531.000000,49261.000000,539524.000000,791746.000000
8,Argentina,1366182.000000,36902.000000,1187053.000000,142227.000000
9,Colombia,1240493.000000,35104.000000,1144923.000000,60466.000000


In [31]:
""" Covidly Project
    CS 4390: Information Retrieval
    @Team 2
    Description: This file is used to create and render the plotly maps used in the frontend.
"""
import json
from urllib.request import urlopen

import plotly.graph_objs as go
from plotly.offline import plot

from . import getdata

ImportError: attempted relative import with no known parent package

In [29]:
date, time = df.iloc[0,4].split()
#print(date,time)
fig_map2 = px.scatter_mapbox(
    df, lat="Lat", lon="Long_",
    color="Deaths",
    size="Deaths",
    color_continuous_scale=px.colors.sequential.Jet,
    size_max=60,
    zoom=3, 
    center={"lat": 37.0902, "lon": -95.7129},              
    hover_data=['Province_State'],
    mapbox_style='open-street-map',
    hover_name='Country_Region'
)
fig_map2.update_layout(
    title=f'COVID-19 Deaths in USA by states.  Date: {date}',
    #mapbox_layers=[
        #{
       #     "below": 'traces',
      #      "sourcetype": "raster",
     #       "sourceattribution": "United States Geological Survey",
    #        "source": [
   #             "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
  #          ]
 #       }
#      ]
)
plot(fig_map2)

'temp-plot.html'

In [29]:
def usa_map():
    with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
        counties = json.load(response)

    df = getdata.usa_counties()
    df.drop([2311], inplace=True)

    fig = go.Figure(
        go.Choroplethmapbox(
            geojson = counties, 
            locations = df.fips,
            z = df['cases/capita'],
            marker_opacity = 0.75,
            marker_line_width = 0,
            colorscale = [[0, '#FFFAF4'], [.005, '#FFE4CC'], [.030, '#DC654F'], [.060, '#CA3328'], [.080, '#B80000'], [.100, '#7C100C'], [.150, '#580000'], [.175, '#300000'], [1, '#170707']]
        )
    )

    fig.update_layout(
        mapbox_style = 'carto-positron', 
        paper_bgcolor='rgba(0,0,0,0)', 
        mapbox_zoom=2.75, 
        mapbox_center = {'lat': 37.0902, 'lon': -95.7129}, 
        margin = dict(t=0, l=0, r=0, b=0)
    )
    plot_div = plot(fig, include_plotlyjs=False, output_type='div', config={'displayModeBar': False})

    return plot_div

In [30]:
plot(usa_map())

NameError: name 'getdata' is not defined

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
Note: you may need to restart the kernel to use updated packages.
